##### Springboard Unit 5 - API mini project

### Vicki Brown - August 15, 2019

This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - per PEP8, constants should always be named in all upper case
API_KEY = ''

**Delete the API KEY before posting the final notebook file**

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exchange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series.

Further information on the data fed is at https://www.quandl.com/data/FSE-Frankfurt-Stock-Exchange?keyword=AFX_X.

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/.

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).

Also, since you won't have access to DataFrames, you are encouraged to use Python's native data structures - preferably dictionaries, though some questions can also be answered using lists. You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

## Setup

In [2]:
# Import the relevant modules
import collections
import requests
import json


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#
# Quandl product overview: https://www.quandl.com/data/FSE-Frankfurt-Stock-Exchange/documentation
#
# API Usage:
#   https://www.quandl.com/api/v3/datasets/{Quandl_Code}/data.format?parameters&api_key={API_KEY}
#   {Quandl_Code} = FSE/{company_code}_X
#   company_code = AFX
# https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json

quandl_url = 'https://www.quandl.com/api/v3/datasets/'
quandl_code = 'FSE/AFX_X'
data_type = 'json'
parameters = 'start_date=2019-08-01&end_date=2019-08-01'
api_url = f'{quandl_url}{quandl_code}/data.{data_type}?{parameters}&api_key={API_KEY}' 

r = requests.get(api_url)

# r.status_code == 200
r.status_code == requests.codes.ok

True

## Examine the data

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

print(json.dumps(r.json(), indent=2))

{
  "dataset_data": {
    "limit": null,
    "transform": null,
    "column_index": null,
    "column_names": [
      "Date",
      "Open",
      "High",
      "Low",
      "Close",
      "Change",
      "Traded Volume",
      "Turnover",
      "Last Price of the Day",
      "Daily Traded Units",
      "Daily Turnover"
    ],
    "start_date": "2019-08-01",
    "end_date": "2019-08-01",
    "frequency": "daily",
    "data": [
      [
        "2019-08-01",
        99.6,
        100.7,
        99.3,
        100.5,
        null,
        102399.0,
        10267086.0,
        null,
        null,
        null
      ]
    ],
    "collapse": null,
    "order": null
  }
}


### Description

The JSON object is a nested dictionary.
   * The outermost layer is a dictionary with one item
      * `dataset_data : {another dictionary}`

   * The next dictionary layer has 10 items:
      * 8 simple `key : value` pairs referencing metadata from the call; the values are strings.
      * 1 pair (key: `"column_names"`) where the value is a _list_ of column names. 
      * 1 pair (key: `"data"`) where the value is a _list of lists_ (a 2-D list).

This `data` list of lists contains the actual data returned from the call. The elements in the sublists map one-to-one with the list keyed by `"column names"`. Each item in the `data` list contains the data for a given date. 

e.g., `r_dict["dataset_data"]["data"][0]`, contains the data for the first day returned.
 
------------------
**Structure** (flattened and simplified)
        
```
{
  "dataset_data": {
    "limit": value,
    "transform": value
    "column_index": value,
    "column_names": ["Date", "Open", "High", "Low", ..., "Daily Turnover"],
    "start_date": "YYYY-MM-DD",
    "end_date": "YYYY-MM-DD",
    "frequency": value,
    "data": [
      [ "YYYY-MM-DD", value, value, value, ..., value ],
      [ "YYYY-MM-DD", value, value, value, ..., value ]
    ],
    "collapse": value,
    "order": value
  }
}
```


### Practice retrieving things from the data

In [5]:
r_dict = r.json()
for key in r_dict["dataset_data"]:
    print(key)

limit
transform
column_index
column_names
start_date
end_date
frequency
data
collapse
order


In [6]:
print( r_dict["dataset_data"]["column_names"] )

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [7]:
print( r_dict["dataset_data"]["column_names"][3] )

Low


In [8]:
print( r_dict["dataset_data"]["data"][0] )

['2019-08-01', 99.6, 100.7, 99.3, 100.5, None, 102399.0, 10267086.0, None, None, None]


In [9]:
print( r_dict["dataset_data"]["data"][0][4] )

100.5


In [10]:
d = r_dict["dataset_data"]["data"]

print(d[0][0])
print(d[0][4])

2019-08-01
100.5


In [11]:
i_close = r_dict["dataset_data"]["column_names"].index('Close')

print(d[0][i_close])

100.5


## Mini Project

### Tasks

These are your tasks for this mini project:

1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).


In [12]:
# These haven't changed
# quandl_url = 'https://www.quandl.com/api/v3/datasets/'
# quandl_code = 'FSE/AFX_X'
# data_type = 'json'

parameters = 'start_date=2017-01-01&end_date=2017-12-31'
api_url = f'{quandl_url}{quandl_code}/data.{data_type}?{parameters}&api_key={API_KEY}' 

r_afx = requests.get(api_url)
r_afx.status_code == requests.codes.ok

True

#### 2. Convert the returned JSON object into a Python dictionary.

In [13]:
# When using requests, the type of the return value of .json() is a dictionary
afx_dict = r_afx.json()
print(type(afx_dict))

<class 'dict'>


In [14]:
# Examine an arbitrary row of the data
print( afx_dict["dataset_data"]["data"][4] )

['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None]


**Planning**: 

Looking forward, we'e going to need to keep dipping into the data rows for different values, e.g. opening prices,
closing price, high and low...

1. We could use the list `.index()` method to get numeric indexes based on desired column names, (e.g. "Open"), then walk through the list of lists of `data`, gathering the value from that position for every row (i.e. date).

2. Alternatively, we could map the `column_names` to the elements in each date's data list, as a dicionary

Option 2 seems cleaner, easier to use, easier to read, and a better general solution.

There are multiple ways to build the map:
1. as a list of dicts, indexed by an integer, one list element per day
2. as a dict of dicts, with the outer dictionary keyed by the date
3. as a list of tuples...

as a list of dictionaries
```
[{ 'Date' : "2019-08-01", 'Open' : 99.6, 'High' : 100.7, 'Low' : 99.3, ...},
 { 'Date' : "2019-08-02", 'Open' : 99.0, 'High' : 100.1, 'Low' : 99.0, ...},
...]
```

as a nested dictionary
```
{'2019-08-01': {'Open': 99.6, 'High': 100.7, 'Low': 99.3, ...},
 '2019-08-02': {'Open': 99.0, 'High': 104, 'Low': 98, ...},
...}
```

I like the dict of dicts approach.

In [15]:
# Extract the list of column names from the original request data, skipping the "Date" column
column_names = afx_dict["dataset_data"]["column_names"][1:]

# Extract the data portion as a list of lists, ensuring that it's sorted by date
data_list = sorted(afx_dict["dataset_data"]["data"])

# Initialize a new dictionary
data_dict = {}

# and populate it, using the date field from each data row as the key
for row in data_list:
    row_date = row[0]
    data_dict[row_date] = collections.OrderedDict(zip(column_names, row[1:]))      


I printed data_dict. It's large, so I've exceroted it here.

```
data_dict

{'2017-01-02': OrderedDict([('Open', 34.99),
              ('High', 35.94),
              ('Low', 34.99),
              ('Close', 35.8),
              ('Change', None),
              ('Traded Volume', 44700.0),
              ('Turnover', 1590561.0),
              ('Last Price of the Day', None),
              ('Daily Traded Units', None),
              ('Daily Turnover', None)]),
 '2017-01-03': OrderedDict([('Open', 35.9),
              ('High', 35.93),
              ...]),
 '2017-12-29': OrderedDict([('Open', 51.76),
              ('High', 51.94),
              ('Low', 51.45),
              ('Close', 51.76),
              ('Change', None),
              ('Traded Volume', 34640.0),
              ('Turnover', 1792304.0),
              ('Last Price of the Day', None),
              ('Daily Traded Units', None),
              ('Daily Turnover', None)])}
 ```

In [16]:
# Compare the new dictionary to the original JSON data structure.

# original JSON data structure
print( afx_dict["dataset_data"]["column_names"] )
print( afx_dict["dataset_data"]["data"][4] )

print("\n")

# This should have the same data
print(data_dict['2017-12-21'])


['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None]


OrderedDict([('Open', 51.16), ('High', 51.52), ('Low', 50.9), ('Close', 51.4), ('Change', None), ('Traded Volume', 120649.0), ('Turnover', 6179433.0), ('Last Price of the Day', None), ('Daily Traded Units', None), ('Daily Turnover', None)])


#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [17]:
# Get a list of Opening prices per day
opening_by_day = [data_dict[day]['Open'] 
                  for day in data_dict 
                  if data_dict[day]['Open'] != None
                 ]

highest_open = max(opening_by_day)
lowest_open = min(opening_by_day)

print('The highest opening price was', highest_open)
print('The lowest opening price was', lowest_open)

The highest opening price was 53.11
The lowest opening price was 34.0


#### 4. What was the largest change in any one day (based on High and Low price)?

In [18]:
# Get a list of price changes (high - low) per day
change_by_day = [(data_dict[day]['High'] - data_dict[day]['Low']) 
                 for day in data_dict 
                 if (data_dict[day]['High'] != None and data_dict[day]['Low'] != None)
                ]

print("The largest change in any one day was", round(max(change_by_day),3))

The largest change in any one day was 2.81


#### 5. What was the largest change between any two days (based on Closing Price)?

Because we sorted the original data list of lists and used collections.OrderedDict to create the dictionary, we know that we have our data in date order.


In [19]:
# Get the list of closing prices
closing_by_day = [data_dict[day]['Close'] 
                  for day in data_dict 
                 if data_dict[day]['Close'] != None
                 ]

#It's easier to compare to the previous day than the next day; there's always a yesterdsy 
previous = closing_by_day.pop(0)

max_change = 0 # initialize

# loop over the list, starting with the 2nd day in the oiginal list
# calculating change from previous day's price to today's price
for price in closing_by_day:
    change = round(abs(price - previous),3) # Avoid 17 "significant" digits
    max_change = max(change, max_change)
    previous = price

print("The largest change in closing price between any two days was", max_change)

The largest change in closing price between any two days was 2.56


#### 6. What was the average daily trading volume during this year?

In [20]:
# Get the list of traded_volume, per day
traded_volume = [data_dict[day]['Traded Volume'] 
                  for day in data_dict 
                 if data_dict[day]['Traded Volume'] != None
                 ]
# count the days
num_days=len(traded_volume)

# get the mean
avg_tv=round(sum(traded_volume)/num_days, 1)

print("The average daily trading volume during this year was", avg_tv)

The average daily trading volume during this year was 89124.3


#### 7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

**Note**: The statistics module is included in the standard library for Python 3.

But since you asked, and since it's so easy, I'll do it both ways.

In [21]:
# test_list=[5,2,5,6,1,2,6,7,2,6,3,5,5]

def do_median(nums):
    """
    usage: do_median(list_of_numbers)
    """
    nums = sorted(nums) 
    count = len(nums)
    l = count//2
    if (count % 2):
        # get the middle number
        return(nums[l])
    else:
        # average the two middle numbers
        return((nums[1] + nums[l+1])/2)
        
print("The median trading volume during this year was", do_median(traded_volume))
        

The median trading volume during this year was 76286.0


In [22]:
# Python 3 includes a statistics package
import statistics

print("The median trading volume during this year was", statistics.median(traded_volume))

The median trading volume during this year was 76286.0
